In [1]:
import importlib
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

In [2]:
from analysis_functions import *

In [3]:
importlib.reload(sys.modules['analysis_functions'])
from analysis_functions import *

In [4]:
X_stress_components_new = opener('X_stress_components_new', path_import='./resourses/')
X_strain_components_new = opener('X_strain_components_new', path_import='./resourses/')
y_stress_components_new = opener('y_stress_components_new', path_import='./resourses/')
y_strain_components_new = opener('y_strain_components_new', path_import='./resourses/')
# X_stress_components_other = opener('X_stress_components_other')
# X_strain_components_other = opener('X_strain_components_other')
# y_stress_components_other = opener('y_stress_components_other')
# y_strain_components_other = opener('y_strain_components_other')

./resourses//X_stress_components_new.pkl
./resourses//X_strain_components_new.pkl
./resourses//y_stress_components_new.pkl
./resourses//y_strain_components_new.pkl


In [5]:
X_stress_components_new.shape

(3, 2678, 5)

In [6]:
component_num = 1
n_trials = 200

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

In [8]:
import shutil
import numpy as np
from sklearn.metrics import mean_squared_error

def plot_results(x, y, model_name, do_zip=False, dpi=500, delete_and_rewrite=True):
    cur_X_test, cur_y_test, *_ = get_train_test(x, y)

    model = opener(model_name)
    cur_prediction = model.predict(cur_X_test)
    print('rmse = ', np.sqrt(mean_squared_error(cur_y_test, cur_prediction)))
    
    if delete_and_rewrite:
        os.makedirs(model_name, exist_ok=True)
    
    for i, item in enumerate(cur_X_test[::20]):
        fig = plt.figure(figsize=(5, 4))
        plt.plot(
            cur_X_test[i*20:(i+1)*20, -1],
            cur_prediction[i*20:(i+1)*20],
            '-o',
            label='predicted'
        )
        plt.plot(
            cur_X_test[i*20:(i+1)*20, -1],
            cur_y_test[i*20:(i+1)*20],
            '-o',
            label='real'
        )
        title = f'red {item[0]}, cal {item[1]}, alpha {int(item[2])}, fric {item[3]}'
        plt.title(title)
        plt.legend(loc='best')
        plt.xlabel('r, -')
        plt.ylabel('$\sigma$, MPa')
        # plt.show()
        plt.savefig(''.join((model_name, '/', title, '.png')), format='png', dpi=dpi)
        plt.close()
        # break
    if do_zip: shutil.make_archive(model_name, 'zip', model_name)

In [9]:
X_stress_components_new[0].max(axis=0)

array([ 0.25,  1.  , 20.  ,  0.1 , 40.  ])

# KAN 

In [10]:
import importlib
import sys

KANModelTrainTest = importlib.reload(sys.modules["analysis_functions"]).KANModelTrainTest

In [11]:
kan_model:KANModelTrainTest = KANModelTrainTest() # type: ignore

In [12]:
kan_model.create_train_val_test(X=X_stress_components_new[0], y=y_stress_components_new[0])

In [13]:
kan_model.optimize_hyperparams(n_trials=100, max_n_layers=4, max_n_units=5, max_steps=300)

[I 2025-03-25 23:51:48,752] A new study created in memory with name: no-name-b9f4ec52-279d-41d0-ad3e-5071f6585849
2025-03-25 23:51:48,761 - INFO - width = [6, 1, 2, 1]


checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                          | 0/158 [01:42<?, ?it/s]
2025-03-25 23:53:39,419 - ERROR - Error during training: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 5619001600 bytes.
[I 2025-03-25 23:53:49,282] Trial 0 finished with value: 1000000.0 and parameters: {'n_layers': 2, 'opt': 'Adam', 'steps': 158, 'n_units_0': 1, 'n_units_1': 2}. Best is trial 0 with value: 1000000.0.
2025-03-25 23:53:49,535 - INFO - width = [6, 5, 1]


checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                           | 0/12 [02:34<?, ?it/s]
2025-03-25 23:56:40,060 - ERROR - Error during training: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 5619001600 bytes.
[I 2025-03-25 23:56:53,488] Trial 1 finished with value: 1000000.0 and parameters: {'n_layers': 1, 'opt': 'LBFGS', 'steps': 12, 'n_units_0': 5}. Best is trial 0 with value: 1000000.0.
2025-03-25 23:56:53,721 - INFO - width = [6, 1, 4, 5, 2, 1]


...

checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                          | 0/182 [01:49<?, ?it/s]
2025-03-26 03:40:02,083 - ERROR - Error during training: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 5619001600 bytes.
[I 2025-03-26 03:40:04,692] Trial 98 finished with value: 1000000.0 and parameters: {'n_layers': 1, 'opt': 'LBFGS', 'steps': 182, 'n_units_0': 5}. Best is trial 0 with value: 1000000.0.
2025-03-26 03:40:04,997 - INFO - width = [6, 1, 3, 1]


checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                           | 0/17 [01:32<?, ?it/s]
2025-03-26 03:41:46,866 - ERROR - Error during training: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 5619001600 bytes.
[I 2025-03-26 03:41:52,656] Trial 99 finished with value: 1000000.0 and parameters: {'n_layers': 2, 'opt': 'Adam', 'steps': 17, 'n_units_0': 1, 'n_units_1': 3}. Best is trial 0 with value: 1000000.0.


{'n_layers': 2, 'opt': 'Adam', 'steps': 158, 'n_units_0': 1, 'n_units_1': 2}

In [23]:
(kan_model.input_layer, kan_model.output_layer,)


(6, 1)

In [30]:
len(kan_model.output_layer.shape)

1